In [1]:
import pandas as pd

In [2]:
%ls

데이터_전처리_2.ipynb
데이터 전처리.ipynb


In [7]:
filename = '서울시 우리마을가게 상권분석서비스(상권-추정매출).csv'
df = pd.read_csv(filename, encoding='cp949')
df.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2020,4,U,관광특구,1001496,강남 마이스 관광특구,CS300043,전자상거래업,3808746,170,...,0,39,131,0,5,116,49,0,0,8
1,2020,4,U,관광특구,1001496,강남 마이스 관광특구,CS300032,가전제품,2123968546,8062,...,210,3726,3652,84,2400,2154,1469,850,418,8
2,2020,4,U,관광특구,1001496,강남 마이스 관광특구,CS300031,가구,11920008477,6667,...,0,2753,3914,0,306,634,1455,1520,2752,6
3,2020,4,U,관광특구,1001496,강남 마이스 관광특구,CS300028,화초,409610564,9795,...,12,5050,4262,30,2543,3556,1490,930,763,6
4,2020,4,U,관광특구,1001496,강남 마이스 관광특구,CS300027,섬유제품,971389704,31185,...,856,5768,25417,1111,15614,6853,3873,3166,568,6


## 상권코드명으로부터 위치 가져오기

In [73]:
names = df['상권_코드_명'].unique()
names

array(['강남 마이스 관광특구', '잠실 관광특구', '종로?청계 관광특구', ..., '돈화문로11가길', '난계로27길',
       '계동길'], dtype=object)

In [1]:
import urllib
import json
import requests

key = 'API KEY' ############# key 발급

In [112]:
def request_json(url):
    response = requests.get(url)
    response_json = json.loads(response.text)
    return response_json

def search(query):
    search_url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={query}&inputtype=textquery&key={key}'
    places_json = request_json(search_url)
    places = places_json['candidates']
    
    if len(places) == 0:
        return
    
    place_id = places[0]['place_id']
    detail_url = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={key}&language=ko'
    response_json = request_json(detail_url)
    
    result = response_json['result']
    geometry = result['geometry']
    location = geometry['location']

    lat = location['lat']
    lng = location['lng']
        
    gu = None
    address_components = result['address_components']
    for component in address_components:
        if 'sublocality_level_1' in component['types']:
            gu = component['long_name']

    return lat, lng, gu

In [113]:
# result 있으면 lat, long, 구 정보 반환
res = search('암사종합시장')
res

(37.55079999999999, 127.1289588, '강동구')

In [114]:
# result 없으면 None 반환
res = search(names[0])
type(res)

NoneType

In [117]:
additional_infos = {}

for name in names:
    res = search(name)
    if res:
        additional_infos[name] = res
    # print(name, res)

강남 마이스 관광특구 None
잠실 관광특구 (37.504832, 127.088065, '송파구')
종로?청계 관광특구 (37.5691015, 126.9786692, '종로구')
동대문패션타운 관광특구 (37.5676999, 127.0088414, '중구')
명동 남대문 북창동 다동 무교동 관광특구 (37.5668175, 126.9816917, '중구')
이태원 관광특구 (37.535001, 126.9928168, '용산구')
길동복조리시장 (37.5388669, 127.1435921, '강동구')
둔촌역전통시장 (37.5279312, 127.1350664, '강동구')
성내골목시장 (37.5349, 127.128065, '강동구')
천호시장 (37.54099250000001, 127.129689, None)
동서울시장 (37.54217939999999, 127.1282889, '강동구')
고분다리골목시장 (37.542838, 127.131387, None)
양지골목시장 (37.58880870000001, 126.9119476, '은평구')
암사종합시장 (37.55079999999999, 127.1289588, '강동구')
고덕전통시장 (37.55913399999999, 127.168512, None)
명일골목시장 (37.5497065, 127.1447593, '강동구')
새마을시장 (37.4739441, 126.8512489, None)
석촌시장 (37.501584, 127.10911, None)
방이시장 (37.5107653, 127.1160681, '송파구')
마천시장 (37.4979629, 127.1501769, '송파구')
풍납시장 (37.5375687, 127.120262, '송파구')
강남개포시장 (37.4885214, 127.0674325, '강남구')
도곡시장 (37.4973378, 127.0516327, '강남구')
청담삼익시장 (37.5219882, 127.0596449, '강남구')
논현종합시장 (37.50995940000001, 127.

제일평화시장 (37.5684231, 127.011236, '중구')
동평화시장 (37.569424, 127.0132438, '중구')
남평화시장 (37.5691284, 127.0108472, '중구')
방산종합시장 (37.5687294, 127.0008004, '중구')
평화시장 (37.56936079999999, 127.0065964, '중구')
인현시장 (37.56312370000001, 126.9951313, '중구')
신중부시장 (37.5651762, 127.0009092, None)
남대문시장 (37.5591786, 126.9776692, '중구')
동묘시장 (37.5733055, 127.017611, '종로구')
창신골목시장 (37.5721798, 127.0124898, '종로구')
동문시장 (33.5115818, 126.526099, None)
충신시장 (37.57503, 127.005419, None)
종로신진시장 (37.5703868, 127.0039905, '종로구')
동대문종합시장 (37.5712417, 127.0049562, None)
광장시장 (37.5701527, 126.9993404, '종로구')
통인시장 (37.5807168, 126.9699822, '종로구')
서울 노원구 노원역_3 (37.6555, 127.06062, '노원구')
서울 노원구 노원역_2 (37.654527, 127.060551, None)
서울 노원구 노원역_1 (37.6563587, 127.063292, '노원구')
창동역 (37.65276, 127.047945, None)
서울 강북구 수유역_3 (37.63886, 127.0231577, '강북구')
등나무근린공원 주변 (37.6404718, 127.0670976, '노원구')
서울 강북구 수유역_2 (37.6371095, 127.0247325, None)
서울 강북구 수유역_1 (37.6371095, 127.0247325, None)
서울 은평구 연신내역_3 (37.61863439999999, 126.920

서울 송파구 석촌역_2 (37.5057138, 127.1070689, None)
선릉역_2 (37.5043, 127.04925, '강남구')
서울 강남구 역삼역_2 (37.49847889999999, 127.0313779, '강남구')
서울 송파구 석촌역_1 (37.5062734, 127.1092586, '송파구')
석촌역 (37.5057138, 127.1070689, None)
서울 강남구 강남역_2 (37.497952, 127.027619, None)
선릉역_1 (37.5046, 127.05012, '강남구')
구로역 (37.50317839999999, 126.8820367, None)
역삼역_2 (37.49847889999999, 127.0313779, '강남구')
대림역_2 (37.49526420000001, 126.8938131, '구로구')
서울 강남구 역삼역_1 (37.4999072, 127.0373932, '강남구')
신대방삼거리역 (37.499717, 126.928189, None)
서울 강남구 한티역 (37.4994914, 127.0610527, '강남구')
은마아파트입구사거리_4 (37.498778, 127.0600406, None)
은마아파트입구사거리_3 (37.498778, 127.0600406, None)
은마아파트입구사거리_2 (37.498778, 127.0600406, None)
오류동역 (37.49440000000001, 126.844551, None)
서울 강남구 강남역 (37.5172363, 127.0473248, '강남구')
은마아파트입구사거리_1 (37.498778, 127.0600406, None)
대림역_1 (37.4948334, 126.9008495, '영등포구')
교대역_4 (37.4947083, 127.0155029, '서초구')
서울 강남구 강남역_1 (37.4981374, 127.0291859, '강남구')
방배동카페골목 (37.4934767, 126.9865297, '서초구')
가락시장 (37.4933411,

방배중앙로21길 (37.49212199999999, 126.9845644, '서초구')
효령로72길 (37.4865408, 127.0268239, '서초구')
효령로34길 (37.48005860000001, 127.000064, '서초구')
효령로31길 (37.4834504, 126.9953655, '서초구')
효령로23길 (37.4823334, 126.9932564, '서초구')
청두곶길 (37.4804987, 126.985628, '서초구')
주흥길 (37.5069166, 127.0195757, '서초구')
주흥15길 (37.50878240000001, 127.0179203, '서초구')
언남길 (37.4727204, 127.0425368, '서초구')
신반포로45길 (37.5117775, 127.0178926, '서초구')
서초대로8길 (37.48365460000001, 126.9883057, '서초구')
서초대로64길 (37.4932577, 127.0213012, '서초구')
서초대로4길 (37.483831, 126.9859554, '서초구')
서초대로25길 (37.4901581, 126.9931003, '서초구')
서초대로23길 (37.49030570000001, 126.9909859, '서초구')
서초대로15길 (37.4885625, 126.989361, '서초구')
사평대로57길 (37.506945, 127.0222083, '서초구')
사평대로26길 (37.4984167, 126.9989049, '서초구')
사평대로22길 (37.4977817, 126.9975644, '서초구')
사임당로17길 (37.4921846, 127.0187788, '서초구')
방배로42길 (37.4944967, 126.9901513, '서초구')
방배로35길 (37.4925271, 126.9880595, '서초구')
방배로18길 (37.48589339999999, 126.9981081, '서초구')
반포대로9길 (37.4828731, 127.0091222, '서초구')
바

시흥대로68길 (37.4575775, 126.902525, '금천구')
시흥대로63길 (37.4553132, 126.8975189, '금천구')
시흥대로62가길 (37.4554253, 126.9027831, '금천구')
시흥대로36길 (37.4434656, 126.9061406, '금천구')
시흥대로22길 (37.4408719, 126.9062836, '금천구')
시흥대로153길 (37.4768068, 126.8952629, '금천구')
시흥대로150길 (37.47636, 126.9007077, '금천구')
시흥대로141길 (37.475232, 126.8975726, '금천구')
시흥대로138길 (37.4723054, 126.899362, '금천구')
시흥대로120길 (37.4687061, 126.9001857, '금천구')
벚꽃로56길 (37.482747, 126.8845978, '금천구')
범안로17길 (37.4680863, 126.8937169, '금천구')
범안로11길 (37.4680761, 126.8911737, '금천구')
문성로5길 (37.4783102, 126.9050001, '금천구')
독산로85길 (37.4723136, 126.9021582, '금천구')
독산로78다길 (37.4698006, 126.9054882, '금천구')
독산로70길 (37.4675247, 126.9048111, '금천구')
독산로64길 (37.4661597, 126.905516, '금천구')
독산로50길 (37.4620301, 126.9092995, '금천구')
독산로40길 (37.4609837, 126.9076563, '금천구')
독산로108길 (37.4796435, 126.9065128, '금천구')
남부순환로112길 (37.4772941, 126.8937309, '금천구')
금하로24길 (37.4484721, 126.9087504, '금천구')
금하로1길 (37.45353, 126.8910694, '금천구')
금하로14길 (37.4518093, 126.903814

매봉산로2길 (37.5773826, 126.8956667, '마포구')
망원로2길 (37.5545778, 126.902117, '마포구')
망원로11길 (37.5595587, 126.9058145, '마포구')
만리재옛길 (37.5475132, 126.9568875, '마포구')
마포대로4길 (37.5368684, 126.9459292, '마포구')
마포대로20길 (37.5518015, 126.9576095, '마포구')
마포대로12길 (37.5470612, 126.954604, '마포구')
동교로38길 (37.5616698, 126.925644, '마포구')
동교로27길 (37.5596674, 126.9217343, '마포구')
동교로25길 (37.5588443, 126.9205348, '마포구')
동교로19길 (37.5575678, 126.9170088, '마포구')
동교로17길 (37.5573668, 126.914952, '마포구')
독막로8길 (37.54686360000001, 126.9179263, '마포구')
독막로32길 (37.546154, 126.938245, '마포구')
독막로22길 (37.5464659, 126.9283471, '마포구')
도화길 (37.5409204, 126.949725, '마포구')
도화4길 (37.5399499, 126.951186, '마포구')
대흥로28길 (37.5552531, 126.9472639, '마포구')
대흥로21길 (37.5532508, 126.9440734, '마포구')
광성로6길 (37.5478241, 126.939555, '마포구')
홍제내길 (37.5867942, 126.9368699, '서대문구')
홍연길 (37.5760587, 126.9298634, '서대문구')
통일로39길 (37.5877256, 126.9417092, '서대문구')
통일로37길 (37.587035, 126.9440977, '서대문구')
통일로11길 (37.5692354, 126.9623247, '서대문구')
충정로6길 (37.

동소문로6길 (37.5884258, 127.0080677, '성북구')
동소문로2길 (37.5885622, 127.0087616, '성북구')
동소문로26가길 (37.5946033, 127.0216602, '성북구')
동소문로15길 (37.5939418, 127.0144375, '성북구')
동소문로13길 (37.592844, 127.0120448, '성북구')
돌곶이로8길 (37.6069158, 127.0636419, '성북구')
돌곶이로41길 (37.6179479, 127.0460784, '성북구')
돌곶이로28길 (37.6124372, 127.0578399, '성북구')
돌곶이로27길 (37.6113382, 127.0536588, '성북구')
돌곶이로22길 (37.6101523, 127.0604017, '성북구')
돌곶이로14길 (37.608624, 127.0613878, '성북구')
길음로7길 (37.6036883, 127.0226004, '성북구')
중랑천로10길 (37.5922053, 127.0757101, '중랑구')
용마산로94길 (37.5886208, 127.099017, '중랑구')
용마산로118길 (37.6011222, 127.105908, '중랑구')
용마산로112길 (37.5986122, 127.1031192, '중랑구')
용마산로100길 (37.590794, 127.100697, '중랑구')
신내로7가길 (37.6025976, 127.0942105, '중랑구')
사가정로54길 (37.5789821, 127.0874945, '중랑구')
사가정로42길 (37.5787398, 127.0824743, '중랑구')
봉화산로56길 (37.6043989, 127.0979026, '중랑구')
봉화산로4길 (37.5994082, 127.07529, '중랑구')
봉화산로30길 (37.5992809, 127.0852079, '중랑구')
봉우재로33길 (37.5942987, 127.088409, '중랑구')
면목로78길 (37.5907755, 127.0903

이태원로27길 (37.5358037, 126.9938734, '용산구')
이촌로22길 (37.5251695, 126.9561089, '용산구')
원효로89길 (37.5408775, 126.9682014, '용산구')
원효로53길 (37.5372549, 126.9619955, '용산구')
원효로41길 (37.5353044, 126.959382, '용산구')
원효로19길 (37.5333071, 126.9540073, '용산구')
우사단로4길 (37.5312638, 126.9991016, '용산구')
우사단로14길 (37.534102, 126.997323, '용산구')
우사단로10길 (37.5315717, 127.000631, '용산구')
신흥로20길 (37.5468614, 126.9843197, '용산구')
소월로20길 (37.5448662, 126.9847998, '용산구')
서빙고로71길 (37.5223526, 126.9956626, '용산구')
백범로90길 (37.5374207, 126.969245, '용산구')
백범로79길 (37.5412058, 126.9645332, '용산구')
두텁바위로1길 (37.5476149, 126.9754246, '용산구')
대사관로34길 (37.5312026, 127.0057683, '용산구')
녹사평대로46길 (37.5374041, 126.9889937, '용산구')
녹사평대로40나길 (37.53643479999999, 126.988913, '용산구')
녹사평대로32길 (37.5328424, 126.9904918, '용산구')
녹사평대로26길 (37.5313169, 126.9931554, '용산구')
녹사평대로11길 (37.5221978, 126.99317, '용산구')
퇴계로90길 (37.5633744, 127.022044, '중구')
퇴계로8길 (37.5572997, 126.9801569, '중구')
퇴계로56길 (37.5623827, 127.0056361, '중구')
퇴계로46길 (37.5617288, 126.99996

In [118]:
print(additional_infos)

{'잠실 관광특구': (37.504832, 127.088065, '송파구'), '종로?청계 관광특구': (37.5691015, 126.9786692, '종로구'), '동대문패션타운 관광특구': (37.5676999, 127.0088414, '중구'), '명동 남대문 북창동 다동 무교동 관광특구': (37.5668175, 126.9816917, '중구'), '이태원 관광특구': (37.535001, 126.9928168, '용산구'), '길동복조리시장': (37.5388669, 127.1435921, '강동구'), '둔촌역전통시장': (37.5279312, 127.1350664, '강동구'), '성내골목시장': (37.5349, 127.128065, '강동구'), '천호시장': (37.54099250000001, 127.129689, None), '동서울시장': (37.54217939999999, 127.1282889, '강동구'), '고분다리골목시장': (37.542838, 127.131387, None), '양지골목시장': (37.58880870000001, 126.9119476, '은평구'), '암사종합시장': (37.55079999999999, 127.1289588, '강동구'), '고덕전통시장': (37.55913399999999, 127.168512, None), '명일골목시장': (37.5497065, 127.1447593, '강동구'), '새마을시장': (37.4739441, 126.8512489, None), '석촌시장': (37.501584, 127.10911, None), '방이시장': (37.5107653, 127.1160681, '송파구'), '마천시장': (37.4979629, 127.1501769, '송파구'), '풍납시장': (37.5375687, 127.120262, '송파구'), '강남개포시장': (37.4885214, 127.0674325, '강남구'), '도곡시장': (37.4973378, 127.0516327, '강남구'),

In [2]:
def get_gu(df):
    if df['상권_코드_명'] in additional_infos:
        return additional_infos[df['상권_코드_명']][2]
    else:
        return
    
def get_lat(df): # 위도
    if df['상권_코드_명'] in additional_infos:
        return additional_infos[df['상권_코드_명']][0]
    else:
        return
    
def get_lng(df): # 경도
    if df['상권_코드_명'] in additional_infos:
        return additional_infos[df['상권_코드_명']][1]
    else:
        return

In [136]:
df['구'] = df.apply(get_gu, axis=1)

In [138]:
df['위도'] = df.apply(get_lat, axis=1)

In [139]:
df['경도'] = df.apply(get_lng, axis=1)

In [141]:
df

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수,구,위도,경도
0,2020,4,U,관광특구,1001496,강남 마이스 관광특구,CS300043,전자상거래업,3808746,170,...,0,5,116,49,0,0,8,None,NaN,NaN
1,2020,4,U,관광특구,1001496,강남 마이스 관광특구,CS300032,가전제품,2123968546,8062,...,84,2400,2154,1469,850,418,8,None,NaN,NaN
2,2020,4,U,관광특구,1001496,강남 마이스 관광특구,CS300031,가구,11920008477,6667,...,0,306,634,1455,1520,2752,6,None,NaN,NaN
3,2020,4,U,관광특구,1001496,강남 마이스 관광특구,CS300028,화초,409610564,9795,...,30,2543,3556,1490,930,763,6,None,NaN,NaN
4,2020,4,U,관광특구,1001496,강남 마이스 관광특구,CS300027,섬유제품,971389704,31185,...,1111,15614,6853,3873,3166,568,6,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132316,2020,1,A,골목상권,1000001,계동길,CS100005,제과점,257814129,27092,...,234,4316,7089,6188,4351,2989,4,종로구,37.580942,126.986728
132317,2020,1,A,골목상권,1000001,계동길,CS100004,양식음식점,156728306,3589,...,5,1361,1116,331,325,157,3,종로구,37.580942,126.986728
132318,2020,1,A,골목상권,1000001,계동길,CS100003,일식음식점,55484866,1272,...,0,134,245,275,314,162,3,종로구,37.580942,126.986728
132319,2020,1,A,골목상권,1000001,계동길,CS100002,중식음식점,25767161,1413,...,145,368,240,368,96,69,1,종로구,37.580942,126.986728


In [149]:
df['구'].unique()

array([None, '송파구', '종로구', '중구', '용산구', '강동구', '은평구', '강남구', '서초구', '남구',
       '중랑구', '관악구', '동작구', '영등포구', '금천구', '구로구', '강서구', '양천구', '마포구',
       '서대문구', '노원구', '도봉구', '강북구', '성북구', '동대문구', '광진구', '성동구'],
      dtype=object)

In [151]:
df['구'].nunique()

26

In [ ]:
df.to_csv('상권데이터_전처리.csv', encoding='cp949', index=False)